# Preprocessing Python Notebook

Use this notebook to preprocess your cell expression matrix into the format required for GraphComm. The notebook will output the following files:
- nodes.csv
- interactions.csv
- meta.csv
- matrix.csv
- LR_nodes.csv
- Omnipath_network.csv

Please ensure that the input matrix is in the format of cells x genes, and that the column names are the cell names and the row names are the gene names. The meta file should contain the cell names and their corresponding labels. The LR_nodes file should contain the ligand and receptor information, and the Omnipath_network file should contain the interactions between the ligands and receptors.

# Setting the input matrix and folder name

Use this cell to set the name of the input matrix you want to preprocess. The matrix file should be placed in a new folder you created in the "Raw_Data" directory under "Data". The name of this folder will be used as the folder name for the output files. The output files will be saved in the "Preprocessed_Data" directory under "Data". Please set the `input_matrix_name` and `folder_name` variables accordingly.


In [6]:
# Change these variables to match your input matrix and folder name
input_matrix_name = "matrix.csv"
folder_name = "Drosophila"

# DO NOT EDIT THE FOLLOWING PATHS, THEY ARE DERIVED FROM THE ABOVE VARIABLES AND ARE USED TO LOAD AND SAVE THE DATA
input_matrix_path = "../../data/Raw_Data/" + folder_name + "/" + input_matrix_name  # DO NOT EDIT
preprocessed_folder_path = "../../data/Preprocessed_Data/" + folder_name  # DO NOT EDIT

# Importing Libraries and Setting Up Environment

* We import necessary libraries for data analysis (`pandas`, `anndata`, `scanpy`), visualization (`matplotlib`), and potentially PyTorch Geometric (if the installation commands are uncommented). 
* Warnings are suppressed using `warnings.filterwarnings('ignore')`.


In [7]:
from utils import *
import pandas as pd # Use modin for parallel processing
import anndata as ad
import scanpy as sc
import warnings
import os
import datatable as dt
import torch

warnings.filterwarnings('ignore')

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# Use all available threads and cores
os.environ["OMP_NUM_THREADS"] = str(os.cpu_count())
os.environ["OPENBLAS_NUM_THREADS"] = str(os.cpu_count())
os.environ["MKL_NUM_THREADS"] = str(os.cpu_count())

# Set scanpy to use all available cores
sc.settings.n_jobs = -1 

# Helper function for visualization.
%matplotlib inline

2.2.1+cu121



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


KeyError: 'PolyCollection:kwdoc'

# Loading and Preprocessing the Data

* The CSV file (`matrix.csv`) containing the gene expression data is loaded into a Pandas DataFrame.
* An AnnData object is created to store and manipulate gene expression data in a structured format, allowing for efficient analysis.
* Scanpy functions are used for preprocessing:
    * `filter_genes`: Removes genes expressed in too few cells (potentially low-quality data).
    * `normalize_total`: Normalizes the data (e.g., total counts per cell).
    * `log1p`: Applies a log transformation to improve the dynamic range of the data.
    * `neighbors`: Computes cell neighborhood relationships, which are essential for downstream analyses.
    * `leiden`: Performs Leiden clustering to identify groups (clusters) of similar cells based on gene expression patterns.

In [ ]:
# Load the CSV data using datatable for faster reading
dt_frame = dt.fread(input_matrix_path)
matrix = dt_frame[:, 1:].to_pandas()

In [ ]:
# Get the gene names from the first column
gene_names = dt_frame[:, 0].to_list()[0]

# Create the AnnData object
adata = ad.AnnData(X=matrix.T)

# Assign gene names to .var_names if rows represent genes
gene_names_str = [str(name) for name in gene_names]
adata.var_names = gene_names_str

In [ ]:
# Preprocessing with Scanpy functions
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
sc.tl.leiden(adata)

# Creating Metadata and Refining the Data Structure

* We create a `meta` DataFrame to store cell metadata (Leiden clusters).
* The gene expression matrix is manipulated (transposed, adjusted).
* Further preprocessing steps involve grouping cells by cluster and performing calculations on the matrix.


In [ ]:
# Create a DataFrame containing cell metadata (Leiden cluster labels)
meta = pd.DataFrame({"cell": adata.obs["leiden"].index.tolist(), "labels": adata.obs["leiden"].tolist()})
meta.index = meta["cell"].tolist()

print(type(meta))

# Update the `adata.obs` attribute with the cell metadata
adata.obs = meta

# Transpose and adjust the matrix DataFrame
matrix = pd.DataFrame(adata.X.transpose(), columns=adata.obs.index.tolist(), index=adata.var.index.tolist())

# More preprocessing (details would be helpful, specific to your analysis goals)
cell_groups = meta['labels'].unique().tolist()  # Get unique cluster labels
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"] == i].index.tolist()  # Get cells in each cluster
    temp_matrix = matrix[cells]  # Subset matrix for each cluster
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(
        axis=1) >= 0].index.tolist())  # Further processing (e.g., identify highly expressed genes)

# Preparing Cell-Type Specific Expression Data

* The `matrix_list` created in the previous section, which likely contains gene expression information aggregated by cell type (cluster), is assigned to the `cell_type_df` variable.
* (If necessary) Explain whether transposing `adata` is required at this stage and why.


In [ ]:
adata = adata.transpose()  # Transpose back for compatibility? 
cell_type_df = matrix_list

# Loading Ligand-Receptor Database

* The LR database (`intercell_nodes.csv` and `intercell_interactions.csv`) is loaded. This database likely contains information about known ligands, receptors, and their interaction pathways.
* Variables `ligands` and `receptors` contain lists of the ligand and receptor identifiers present in the database. 


In [ ]:
nodes = pd.DataFrame({"category": [], "identifier": []})

# LR_nodes represent ligands and receptors
LR_nodes = pd.read_csv("../../data/LR_database/intercell_nodes.csv", index_col=0)
# Omnipath_network represents known interactions between them
Omnipath_network = pd.read_csv("../../data/LR_database/intercell_interactions.csv", index_col=0)

ligands = LR_nodes[LR_nodes["category"] == "Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"] == "Receptor"]["identifier"].tolist()

# Identifying Cell Type-Specific Ligands and Receptors

* The code iterates through each cell type (cluster) and compares the expressed genes in that cluster with the known ligands and receptors from the LR database.
* `ligand_list` and `receptor_list` keep track of unique ligands and receptors found across cell types.
* `new_cell_df` stores lists of expressed ligands and receptors for each individual cell type.


In [ ]:
ligand_list = []
receptor_list = []
new_cell_df = {}

for i in cell_type_df.keys():
    # Find ligands and receptors expressed within each cell type
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))

    # Store expressed ligands and receptors for each cell type (cluster)
    new_cell_df[i] = [
        list(set(ligands) & set(cell_type_df[i])),  # Expressed ligands 
        list(set(receptors) & set(cell_type_df[i])),  # Expressed receptors
    ]

for i in new_cell_df.keys():
    # For each key, get the first element of the value list (assumed to be ligands)
    # and append "_Ligand" to each ligand to create a unique identifier
    new_cell_df[i][0] = [j + "_Ligand" for j in new_cell_df[i][0]]

    # Get the second element of the value list (assumed to be receptors)
    # and append "_Receptor" to each receptor to create a unique identifier
    new_cell_df[i][1] = [j + "_Receptor" for j in new_cell_df[i][1]]

# Creating the Nodes DataFrame for GraphComm

* **Consolidating node types:** The code combines different node types (cell groups, ligands, receptors) into a single `nodes` DataFrame. Each node is assigned a category to distinguish its type.
* **Unique identifiers:** Unique identifiers for nodes are created, ensuring consistency and avoiding ambiguity within the graph structure.
* **ID assignment:** Each node is assigned a numerical ID to facilitate referencing within the graph structure.


In [ ]:
ligand_list = list(set(ligand_list))  # Get unique ligands
receptor_list = list(set(receptor_list))  # Get unique receptors

# Rename ligands/receptors to include "Ligand" / "Receptor" suffixes
ligand_list = [i + "_Ligand" for i in ligand_list]
receptor_list = [i + "_Receptor" for i in receptor_list]

# Create the nodes DataFrame 
nodes = pd.concat([
    pd.DataFrame(
        {"category": ["Cell Group"] * len(list(cell_type_df.keys())), "identifier": list(cell_type_df.keys())}),
    pd.DataFrame({"category": ["Ligand"] * len(ligand_list), "identifier": ligand_list}),
    pd.DataFrame({"category": ["Receptor"] * len(receptor_list), "identifier": receptor_list})
])

# Create unique IDs for each node
new_identifier = [row["identifier"] + "_" + row["category"] for index, row in LR_nodes.iterrows()]
LR_nodes["identifier"] = new_identifier

# Add ID column to the nodes DataFrame
nodes["Id"] = range(0, nodes.shape[0])
nodes = nodes[["Id", "category", "identifier"]]
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

# Creating the Interactions DataFrame for GraphComm

* **Interactions within cell groups:** The code defines interactions between cell groups and their expressed ligands and receptors. Each cell group serves as the source, with ligands and receptors expressed by the group being the destinations (targets) of the interactions.
* **Edge Attributes:** Sample weights and edge types (assigning all of them with a value of 1) are added to the `interactions` DataFrame.
* **ID Mapping:** Node names in the interactions DataFrame are mapped to their corresponding IDs in the `nodes` DataFrame to maintain consistency within the graph structure.


In [ ]:
# Reset index of meta for easy access
meta.index = meta["cell"].tolist()

# Create the interactions DataFrame
interactions = pd.DataFrame({"Src": [], "Dst": [], "Weight": [], "edge_type": []})

# Aligning DataFrames with Node IDs
LR_nodes.index = LR_nodes["Id"].tolist()

# (Potentially) increment 'Src' and 'Dst' (Double-check the reason behind this)
Omnipath_network["Src"] += 1
Omnipath_network["Dst"] += 1
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()

# Add interactions between cell groups and their expressed ligands/receptors
source_list = []
dest_list = []
weight_list = []
edge_type_list = []

for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))  # Cell group as source
    dest_list.extend(new_cell_df[i][0])  # Ligands as destinations
    dest_list.extend(new_cell_df[i][1])  # Receptors as destinations
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))  # Sample weight
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))  # Sample edge type

interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

# Map node identifiers to IDs for consistency
nodes.index = nodes["identifier"].tolist()

nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0, nodes.shape[0])

interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

LR_nodes, interactions

In [ ]:
# Prepare data and edit the ground truth for GraphComm
first_data, first_nodes, first_interactions = make_dataset(nodes, interactions, first=False, pathway_encode=False)

# Refining Data for Consistency with GraphComm

This code filters the ligand-receptor database (`LR_nodes` and `Omnipath_network`) to align it with the analyzed dataset. It ensures that only nodes (ligands and receptors) and interactions relevant to the current analysis are included in the data provided to GraphComm.

In [ ]:
# Setting the 'identifier' column as the index for easier referencing 
LR_nodes.index = LR_nodes["identifier"].tolist()

# Keep only ligands and receptors present in your analyzed dataset
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]

# Filter interactions to include only those with sources and destinations present in the filtered LR_nodes
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (
    Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

# Update the lists of ligand and receptor identifiers 
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

# Further refine LR_nodes to include only the remaining ligands and receptors
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]

# Filter Omnipath_network to include interactions consistent with the refined LR_nodes
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (
    Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

# Update the lists of ligand and receptor identifiers (again)
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

# Assign unique numerical IDs to each node in LR_nodes
LR_nodes["Id"] = range(0, LR_nodes.shape[0])

# Set the 'identifier' column back as the index
LR_nodes.index = LR_nodes["identifier"].tolist()

# Replace source node identifiers in Omnipath_network with their corresponding IDs from LR_nodes
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()

# Replace destination node identifiers in Omnipath_network with their corresponding IDs from LR_nodes
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

# Assign a uniform edge type to all interactions (replace 1 with a meaningful value if necessary)
interactions["edge_type"] = 1

# Preparing the Output Directory and Saving Data

These CSV files will serve as the input data for GraphComm.

In [ ]:
# Create the directory for the output files
os.system("mkdir -p " + preprocessed_folder_path)

# Save the data to CSV files in the output directory
nodes.to_csv(preprocessed_folder_path + "/nodes.csv")
interactions.to_csv(preprocessed_folder_path + "/interactions.csv")
meta.to_csv(preprocessed_folder_path + "/meta.csv")
matrix.to_csv(preprocessed_folder_path + "/matrix.csv")
LR_nodes.to_csv(preprocessed_folder_path + "/LR_nodes.csv")
Omnipath_network.to_csv(preprocessed_folder_path + "/Omnipath_network.csv")